In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('S:/Hackathons/TCS/train1.csv')
test = pd.read_csv('S:/Hackathons/TCS/test1.csv')
submission = pd.read_excel('S:/Hackathons/TCS/datasets/submission.xlsx')
project = pd.read_csv('S:/Hackathons/TCS/project1.csv')

# 1.same location or not

In [2]:
# Data Prepeocessing for location

train_loc = pd.DataFrame(train[['can_id','location']])
test_loc = pd.DataFrame(test[['can_id','location']])
proj_loc = pd.DataFrame(project[['pro_id','location']])
proj_loc.columns = ['can_id','location']


temp = pd.concat([train_loc,test_loc])
locations = pd.concat([temp,proj_loc])
locations.columns = ['id','location']
locations.drop_duplicates(inplace = True)

locations.index = locations.id
locations.drop(columns = ['id'],inplace = True)
locations['location'].astype(str)


# function

def is_same_loc(c,p):
    if locations.loc[c].values[0] == locations.loc[p].values[0]:
        return 1
    else :
        return 0


is_same_loc('c1','p3')

1

In [3]:
is_same_loc('c5','p20')

0

# 2.mode

In [4]:
# preprocess data for mode

proj_loc = project[['pro_id','mode']]
proj_loc.index = proj_loc.pro_id
proj_loc.drop(columns = ['pro_id'],inplace = True)


# function

def mode(p):
    if proj_loc.loc[p].values[0] == 'PHYGITAL':
       return 1
    else:
       return 0


mode('p20') 

C:\Users\dines\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1

In [5]:
mode('p3') 

0


# 3.number of common skills

In [7]:


skills_list = pd.read_csv('S:/Hackathons/TCS/skills_list.csv')
skills_list.dropna(inplace = True)



def numb_comm_skills(c,p):
    s1 = set(skills_list.loc[skills_list['name'] == c]['skill'].values[0].split(','))
    s2 = set(skills_list.loc[skills_list['name'] == p]['skill'].values[0].split(','))  
    
    return len(list(s1.intersection(s2)))

numb_comm_skills('c20','p200')



2

# 4.top skills score

There will be more chance of getting hired if the candidate has top demanding skills.

Scoring be like:

0.1 - the candidate have 1 out top 10 skills

0.2 - the candidate have 2 out top 10 skills

likewise ............

In [8]:


top_skills_score = pd.read_csv('S:/Hackathons/TCS/top_skills_score.csv')
top_skills_score.index = top_skills_score.name
top_skills_score.drop(columns = ['name'],inplace = True)


def top_score(c):
    a = top_skills_score.loc[c].values[0]
    return a



top_score('c1')



0.30000000000000004

# 5.id lookup 

First Made a Look up table for candidate and Project is using Matrix Factorization
     
Data used for matrix Factorization is One Hot Encoded Table of both (Candidate_id and Project_id) in index
and skills at columns 

In [9]:


###############################


#5.id lookup 


id_lookup = pd.read_csv('S:/Hackathons/TCS/id_lookup.csv')
id_lookup.index =  id_lookup.name
id_lookup.drop(columns = ['name'],inplace = True)

def create_lookup(c,p):
    a = list(id_lookup.loc[c].values)
    b = list(id_lookup.loc[p].values)
    
    return np.add(a,b)
create_lookup('c20','p200')


array([ 0.11058763, -0.05677603,  0.04089024, -0.09040889, -0.05132164,
        0.05292386])

# 6.euclidian between the lookup

Euclidian distance between Latent Vectors of project and candidate id

From Matrix Factorization. First I used Latent Vector for each project and user id as an embedding on the table
but didn’t give better result. Then I Used latent vector for Candidate and project as point and computer 
Euclidian distance between them

In [10]:


from scipy.spatial import distance


def create_eulc(c,p):
    a = id_lookup.loc[c].values
    b = id_lookup.loc[p].values
    
    return distance.euclidean(a, b)
create_eulc('c20','p200')

0.11956813884817027

# 7 jacard score

jacard score for finding the similarity between the one hot encoded of Candidate and Project.

In [12]:

id_vs_skills_one_hot = pd.read_csv('S:/Hackathons/TCS/one_hot.csv')   #id_vs_skills_one_hot
id_vs_skills_one_hot.index = id_vs_skills_one_hot.name
id_vs_skills_one_hot.drop(columns = ['name'],inplace = True)
id_vs_skills_one_hot.drop(columns = ['All'],inplace = True)
id_vs_skills_one_hot.drop('All',inplace = True)

from sklearn.metrics import jaccard_score
def get_jacard_scores(c,p):
    return jaccard_score(id_vs_skills_one_hot.loc[c].values, id_vs_skills_one_hot.loc[p].values)

c = 'c20'
p = 'p200'

get_jacard_scores(c,p)

0.16666666666666666

In [14]:

def create_df(c_col,p_col):
    temp_df = []
    for c,p in zip(c_col,p_col):
        row = [is_same_loc(c,p)]+ [mode(p)]+ [numb_comm_skills(c,p)]+[top_score(c)]+[create_eulc(c,p)]+[get_jacard_scores(c,p)]
        temp_df.append(row)
    
    columns = ['is_same_loc','mode','numb_comm_skills','top_score','eulc_dist','jacard_score_b/w_skills']
    
    df = pd.DataFrame(temp_df,columns = columns) 
    
    return df



### Train Data Creation

In [15]:
train.dropna(inplace = True)
train_df = create_df(train['can_id'],train['pro_id'])


train_df['result'] = list(train['evaluation_status'])
#train_df.to_csv('S:/Hackathons/TCS/train_main.csv',index = False)
train_df.head()

,is_same_loc,mode,numb_comm_skills,top_score,eulc_dist,jacard_score_b/w_skills,result
0,1,0,1,0.2,0.072502,0.166667,SELECTED
1,0,1,0,0.2,0.111115,0.000000,SELECTED
2,1,0,0,0.1,0.067958,0.000000,SELECTED
3,1,0,0,0.2,0.084265,0.000000,SELECTED
4,1,1,1,0.4,0.105762,0.200000,REJECTED


In [46]:
test.isnull().sum()

can_id      0
location    0
skills      2
skill1      2
dtype: int64

In [19]:
test.dropna(inplace = True)
test_pro = []
test_can = []
for i in submission['Submission ']:
    for j in test['can_id']:
        test_pro.append(i)
        test_can.append(j)
        

test.dropna(inplace = True)
test_main = create_df(test_can,test_pro)


test_main['proj_id'] = test_pro
test_main['cand_id'] = test_can

#test_main.to_csv('S:/Hackathons/TCS/test_main1.csv',index = False)

In [17]:
test_main.head()

,is_same_loc,mode,numb_comm_skills,top_score,eulc_dist,jacard_score_b/w_skills,proj_id,cand_id
0,0,0,1,0.3,0.080969,0.142857,p10,c1
1,1,0,0,0.1,0.129904,0.000000,p10,c3
2,1,0,1,0.2,0.057331,0.125000,p10,c7
3,0,0,1,0.7,0.127057,0.076923,p10,c15
4,0,0,1,0.6,0.135926,0.090909,p10,c16
